In [1]:
import os
import glob
import json
import pandas as pd

In [2]:
root = 'C:/Users/naman/Documents/Code/Project/NCKH_2024'
des_path = root + '/dict'

In [5]:
data_root = root + '/data_image/Keyframes'
scene_root = root + '/dict/SceneJSON'
metadata_root = root + '/dict/metadata'
map_keyframes_root = root + '/dict/map_keyframes'
fps_path = root + '/dict/fps.json'

with open(fps_path, 'r') as f:
    fps_dict = json.load(f)

In [6]:
scene_id2info = dict()

for data_part in sorted(os.listdir(scene_root)):
    data_part_path = f'{scene_root}/{data_part}'
    scene_id2info[data_part] = dict()
    for video_path in sorted(os.listdir(data_part_path)):
        video_id = video_path.split('.')[0]
        video_path_full = f'{data_part_path}/{video_path}'
        video_fps = fps_dict[f'{data_part}_{video_id}']
        
        with open(video_path_full, 'r') as f:
            video_scene_info = json.load(f)
            
        with open(f'{metadata_root}/{data_part}_{video_id}.json', 'r', encoding="utf8") as f:
            video_metadata = json.load(f)
        
        scene_id2info[data_part][video_id] = {
            'video_metadata': video_metadata,
            'lst_shot': dict()
        }
        
        for i, item in enumerate(video_scene_info):
            scene_id2info[data_part][video_id]['lst_shot'][str(i)] = {
                'shot_range': item,
                'shot_time': [item[0]/video_fps, item[1]/video_fps],
                'lst_keyframe_paths': [],
                'lst_keyframe_idxs': []
            }

In [11]:
global_index = 0
id2img_fps = []

for data_part in sorted(os.listdir(data_root)):
    data_part_path = f'{data_root}/{data_part}'

    for video_id in sorted(os.listdir(data_part_path)):
        video_dir = f'{data_part_path}/{video_id}'
        
        temp_data_part = data_part.replace('_extra', '') # replace L01_extra -> L01
        with open(f'{scene_root}/{temp_data_part}/{video_id}.json', 'r') as f:
            video_scene_info = json.load(f)
            
        if 'extra' not in data_part:
            mapper = pd.read_csv(f'{map_keyframes_root}/{data_part}_{video_id}.csv', index_col='n')
        
        scene_track = 0
        for image_path in sorted(os.listdir(video_dir)):
            frame_idx = int(image_path.split('.')[0])
            
            if 'extra' not in data_part:
                frame_idx = int(mapper.loc[frame_idx]['frame_idx'])
            
            image_path = f'{video_dir}/{image_path}'.replace(f'{root}/data_image', '')
            
            while(len(video_scene_info) > 0 and frame_idx > video_scene_info[0][1]):
                video_scene_info.pop(0)
                scene_track += 1
            
            if len(video_scene_info) == 0:
                continue
            
            info = {
                "image_path": image_path,
                "scene_idx": f'{temp_data_part}/{video_id}/lst_shot/{str(scene_track)}'
            }
            
            scene_id2info[temp_data_part][video_id]['lst_shot'][str(scene_track)]['lst_keyframe_paths'].append(image_path)
            scene_id2info[temp_data_part][video_id]['lst_shot'][str(scene_track)]['lst_keyframe_idxs'].append(global_index)
            
            id2img_fps.append(info)
            
            global_index += 1

In [12]:
id2img_fps = dict(enumerate(id2img_fps))

In [13]:
with open(f'{des_path}/scene_id2info.json', 'w') as f:
    f.write(json.dumps(scene_id2info))
    
with open(f'{des_path}/id2img_fps.json', 'w') as f:
    f.write(json.dumps(id2img_fps))

print(f'Saved {des_path}')
print(f"Number of Index: {len(id2img_fps)}")

Saved C:/Users/naman/Documents/Code/Project/NCKH_2024/dict
Number of Index: 134685
